# Anna KaRNNa

In this notebook, I'll build a character-wise RNN trained on Anna Karenina, one of my all-time favorite books. It'll be able to generate new text based on the text from the book.

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Also, some information [here at r2rt](http://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html) and from [Sherjil Ozair](https://github.com/sherjilozair/char-rnn-tensorflow) on GitHub. Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

First we'll load the text file and convert it into integers for our network to use.

In [2]:
with open('anna.txt', 'r') as f:
    text=f.read()
vocab = set(text)
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
chars = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

In [4]:
chars[:100]

array([75,  0, 61,  1, 39, 28, 60,  4,  7, 22, 22, 22, 40, 61,  1,  1, 11,
        4, 55, 61, 57, 72,  5, 72, 28, 68,  4, 61, 60, 28,  4, 61,  5,  5,
        4, 61,  5, 72, 12, 28, 21,  4, 28, 43, 28, 60, 11,  4, 24, 51,  0,
       61,  1,  1, 11,  4, 55, 61, 57, 72,  5, 11,  4, 72, 68,  4, 24, 51,
        0, 61,  1,  1, 11,  4, 72, 51,  4, 72, 39, 68,  4, 64, 33, 51, 22,
       33, 61, 11, 25, 22, 22, 66, 43, 28, 60, 11, 39,  0, 72, 51],
      dtype=int32)

Now I need to split up the data into batches, and into training and validation sets. I should be making a test set here, but I'm not going to worry about that. My test will be if the network can generate new text.

Here I'll make both input and target arrays. The targets are the same as the inputs, except shifted one character over. I'll also drop the last bit of data so that I'll only have completely full batches.

The idea here is to make a 2D matrix where the number of rows is equal to the number of batches. Each row will be one long concatenated string from the character data. We'll split this data into a training set and validation set using the `split_frac` keyword. This will keep 90% of the batches in the training set, the other 10% in the validation set.

In [5]:
def split_data(chars, batch_size, num_steps, split_frac=0.9):
    """ 
    Split character data into training and validation sets, inputs and targets for each set.
    
    Arguments
    ---------
    chars: character array
    batch_size: Size of examples in each of batch
    num_steps: Number of sequence steps to keep in the input and pass to the network
    split_frac: Fraction of batches to keep in the training set
    
    
    Returns train_x, train_y, val_x, val_y
    """
    
    slice_size = batch_size * num_steps
    n_batches = int(len(chars) / slice_size)
    
    # Drop the last few characters to make only full batches
    x = chars[: n_batches*slice_size]
    y = chars[1: n_batches*slice_size + 1]
    
    # Split the data into batch_size slices, then stack them into a 2D matrix 
    x = np.stack(np.split(x, batch_size))
    y = np.stack(np.split(y, batch_size))
    
    # Now x and y are arrays with dimensions batch_size x n_batches*num_steps
    
    # Split into training and validation sets, keep the virst split_frac batches for training
    split_idx = int(n_batches*split_frac)
    train_x, train_y= x[:, :split_idx*num_steps], y[:, :split_idx*num_steps]
    val_x, val_y = x[:, split_idx*num_steps:], y[:, split_idx*num_steps:]
    
    return train_x, train_y, val_x, val_y

In [6]:
train_x, train_y, val_x, val_y = split_data(chars, 10, 200)

In [7]:
train_x.shape

(10, 178400)

In [8]:
train_x[:,:10]

array([[75,  0, 61,  1, 39, 28, 60,  4,  7, 22],
       [15, 51, 58,  4,  0, 28,  4, 57, 64, 43],
       [ 4, 76, 61, 39, 76,  0, 72, 51,  6,  4],
       [64, 39,  0, 28, 60,  4, 33, 64, 24,  5],
       [ 4, 39,  0, 28,  4,  5, 61, 51, 58, 44],
       [ 4, 23,  0, 60, 64, 24,  6,  0,  4,  5],
       [39,  4, 39, 64, 22, 58, 64, 25, 22, 22],
       [64,  4,  0, 28, 60, 68, 28,  5, 55, 26],
       [ 0, 61, 39,  4, 72, 68,  4, 39,  0, 28],
       [28, 60, 68, 28,  5, 55,  4, 61, 51, 58]], dtype=int32)

I'll write another function to grab batches out of the arrays made by split data. Here each batch will be a sliding window on these arrays with size `batch_size X num_steps`. For example, if we want our network to train on a sequence of 100 characters, `num_steps = 100`. For the next batch, we'll shift this window the next sequence of `num_steps` characters. In this way we can feed batches to the network and the cell states will continue through on each batch.

In [9]:
def get_batch(arrs, num_steps):
    batch_size, slice_size = arrs[0].shape
    
    n_batches = int(slice_size/num_steps)
    for b in range(n_batches):
        yield [x[:, b*num_steps: (b+1)*num_steps] for x in arrs]

In [16]:
def build_rnn(num_classes, batch_size=50, num_steps=50, lstm_size=128, num_layers=2,
              learning_rate=0.001, grad_clip=5, sampling=False):
        
    if sampling == True:
        batch_size, num_steps = 1, 1

    tf.reset_default_graph()
    
    # Declare placeholders we'll feed into the graph
    with tf.name_scope('inputs'):
        inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
        x_one_hot = tf.one_hot(inputs, num_classes, name='x_one_hot')
    
    with tf.name_scope('targets'):
        targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
        y_one_hot = tf.one_hot(targets, num_classes, name='y_one_hot')
        y_reshaped = tf.reshape(y_one_hot, [-1, num_classes])
    
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    # Build the RNN layers
    with tf.name_scope("RNN_cells"):
        def make_cell():
            lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
            drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
            return drop
        cell = tf.contrib.rnn.MultiRNNCell([make_cell() for _ in range(num_layers)])
    
    with tf.name_scope("RNN_init_state"):
        initial_state = cell.zero_state(batch_size, tf.float32)

    # Run the data through the RNN layers
    with tf.name_scope("RNN_forward"):
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=initial_state)
    
    final_state = state
    
    # Reshape output so it's a bunch of rows, one row for each cell output
    with tf.name_scope('sequence_reshape'):
        seq_output = tf.concat(outputs, axis=1,name='seq_output')
        output = tf.reshape(seq_output, [-1, lstm_size], name='graph_output')
    
    # Now connect the RNN outputs to a softmax layer and calculate the cost
    with tf.name_scope('logits'):
        softmax_w = tf.Variable(tf.truncated_normal((lstm_size, num_classes), stddev=0.1),
                               name='softmax_w')
        softmax_b = tf.Variable(tf.zeros(num_classes), name='softmax_b')
        logits = tf.matmul(output, softmax_w) + softmax_b
        tf.summary.histogram('softmax_w', softmax_w)
        tf.summary.histogram('softmax_b', softmax_b)

    with tf.name_scope('predictions'):
        preds = tf.nn.softmax(logits, name='predictions')
        tf.summary.histogram('predictions', preds)
    
    with tf.name_scope('cost'):
        loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_reshaped, name='loss')
        cost = tf.reduce_mean(loss, name='cost')
        tf.summary.scalar('cost', cost)

    # Optimizer for training, using gradient clipping to control exploding gradients
    with tf.name_scope('train'):
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip)
        train_op = tf.train.AdamOptimizer(learning_rate)
        optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    merged = tf.summary.merge_all()
    
    # Export the nodes 
    export_nodes = ['inputs', 'targets', 'initial_state', 'final_state',
                    'keep_prob', 'cost', 'preds', 'optimizer', 'merged']
    Graph = namedtuple('Graph', export_nodes)
    local_dict = locals()
    graph = Graph(*[local_dict[each] for each in export_nodes])
    
    return graph

## Hyperparameters

Here I'm defining the hyperparameters for the network. The two you probably haven't seen before are `lstm_size` and `num_layers`. These set the number of hidden units in the LSTM layers and the number of LSTM layers, respectively. Of course, making these bigger will improve the network's performance but you'll have to watch out for overfitting. If your validation loss is much larger than the training loss, you're probably overfitting. Decrease the size of the network or decrease the dropout keep probability.

In [17]:
batch_size = 100
num_steps = 100
lstm_size = 512
num_layers = 2
learning_rate = 0.001

## Training

Time for training which is is pretty straightforward. Here I pass in some data, and get an LSTM state back. Then I pass that state back in to the network so the next batch can continue the state from the previous batch. And every so often (set by `save_every_n`) I calculate the validation loss and save a checkpoint.

In [18]:
!mkdir -p checkpoints/anna

In [19]:
epochs = 10
save_every_n = 100
train_x, train_y, val_x, val_y = split_data(chars, batch_size, num_steps)

model = build_rnn(len(vocab), 
                  batch_size=batch_size,
                  num_steps=num_steps,
                  learning_rate=learning_rate,
                  lstm_size=lstm_size,
                  num_layers=num_layers)

saver = tf.train.Saver(max_to_keep=100)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter('./logs/2/train', sess.graph)
    test_writer = tf.summary.FileWriter('./logs/2/test')
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/anna20.ckpt')
    
    n_batches = int(train_x.shape[1]/num_steps)
    iterations = n_batches * epochs
    for e in range(epochs):
        
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for b, (x, y) in enumerate(get_batch([train_x, train_y], num_steps), 1):
            iteration = e*n_batches + b
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: 0.5,
                    model.initial_state: new_state}
            summary, batch_loss, new_state, _ = sess.run([model.merged, model.cost, 
                                                          model.final_state, model.optimizer], 
                                                          feed_dict=feed)
            loss += batch_loss
            end = time.time()
            print('Epoch {}/{} '.format(e+1, epochs),
                  'Iteration {}/{}'.format(iteration, iterations),
                  'Training loss: {:.4f}'.format(loss/b),
                  '{:.4f} sec/batch'.format((end-start)))
            
            train_writer.add_summary(summary, iteration)
        
            if (iteration%save_every_n == 0) or (iteration == iterations):
                # Check performance, notice dropout has been set to 1
                val_loss = []
                new_state = sess.run(model.initial_state)
                for x, y in get_batch([val_x, val_y], num_steps):
                    feed = {model.inputs: x,
                            model.targets: y,
                            model.keep_prob: 1.,
                            model.initial_state: new_state}
                    summary, batch_loss, new_state = sess.run([model.merged, model.cost, 
                                                               model.final_state], feed_dict=feed)
                    val_loss.append(batch_loss)
                    
                test_writer.add_summary(summary, iteration)

                print('Validation loss:', np.mean(val_loss),
                      'Saving checkpoint!')
                #saver.save(sess, "checkpoints/anna/i{}_l{}_{:.3f}.ckpt".format(iteration, lstm_size, np.mean(val_loss)))

Epoch 1/10  Iteration 1/1780 Training loss: 4.4197 9.7581 sec/batch
Epoch 1/10  Iteration 2/1780 Training loss: 4.3807 9.4150 sec/batch
Epoch 1/10  Iteration 3/1780 Training loss: 4.2224 9.3747 sec/batch
Epoch 1/10  Iteration 4/1780 Training loss: 4.6116 10.4519 sec/batch
Epoch 1/10  Iteration 5/1780 Training loss: 4.5086 8.8608 sec/batch
Epoch 1/10  Iteration 6/1780 Training loss: 4.4054 10.4458 sec/batch
Epoch 1/10  Iteration 7/1780 Training loss: 4.3148 8.5908 sec/batch
Epoch 1/10  Iteration 8/1780 Training loss: 4.2338 8.1956 sec/batch
Epoch 1/10  Iteration 9/1780 Training loss: 4.1561 8.0580 sec/batch
Epoch 1/10  Iteration 10/1780 Training loss: 4.0841 8.2688 sec/batch
Epoch 1/10  Iteration 11/1780 Training loss: 4.0200 8.2632 sec/batch
Epoch 1/10  Iteration 12/1780 Training loss: 3.9668 7.9335 sec/batch
Epoch 1/10  Iteration 13/1780 Training loss: 3.9207 8.1641 sec/batch
Epoch 1/10  Iteration 14/1780 Training loss: 3.8811 8.1274 sec/batch
Epoch 1/10  Iteration 15/1780 Training lo

Epoch 1/10  Iteration 119/1780 Training loss: 3.2256 7.9647 sec/batch
Epoch 1/10  Iteration 120/1780 Training loss: 3.2237 8.0433 sec/batch
Epoch 1/10  Iteration 121/1780 Training loss: 3.2220 8.3069 sec/batch
Epoch 1/10  Iteration 122/1780 Training loss: 3.2202 8.7122 sec/batch
Epoch 1/10  Iteration 123/1780 Training loss: 3.2183 8.3309 sec/batch
Epoch 1/10  Iteration 124/1780 Training loss: 3.2165 8.0580 sec/batch
Epoch 1/10  Iteration 125/1780 Training loss: 3.2144 8.1516 sec/batch
Epoch 1/10  Iteration 126/1780 Training loss: 3.2122 8.3992 sec/batch
Epoch 1/10  Iteration 127/1780 Training loss: 3.2102 8.2417 sec/batch
Epoch 1/10  Iteration 128/1780 Training loss: 3.2081 8.1853 sec/batch
Epoch 1/10  Iteration 129/1780 Training loss: 3.2061 8.1615 sec/batch
Epoch 1/10  Iteration 130/1780 Training loss: 3.2041 8.5034 sec/batch
Epoch 1/10  Iteration 131/1780 Training loss: 3.2021 8.3809 sec/batch
Epoch 1/10  Iteration 132/1780 Training loss: 3.1997 8.0107 sec/batch
Epoch 1/10  Iteratio

Epoch 2/10  Iteration 236/1780 Training loss: 2.4546 8.3060 sec/batch
Epoch 2/10  Iteration 237/1780 Training loss: 2.4532 7.8809 sec/batch
Epoch 2/10  Iteration 238/1780 Training loss: 2.4520 7.9339 sec/batch
Epoch 2/10  Iteration 239/1780 Training loss: 2.4504 7.9789 sec/batch
Epoch 2/10  Iteration 240/1780 Training loss: 2.4491 8.0327 sec/batch
Epoch 2/10  Iteration 241/1780 Training loss: 2.4479 8.0968 sec/batch
Epoch 2/10  Iteration 242/1780 Training loss: 2.4467 8.3639 sec/batch
Epoch 2/10  Iteration 243/1780 Training loss: 2.4451 8.9756 sec/batch
Epoch 2/10  Iteration 244/1780 Training loss: 2.4442 8.1578 sec/batch
Epoch 2/10  Iteration 245/1780 Training loss: 2.4428 8.1266 sec/batch
Epoch 2/10  Iteration 246/1780 Training loss: 2.4411 8.0857 sec/batch
Epoch 2/10  Iteration 247/1780 Training loss: 2.4395 8.5903 sec/batch
Epoch 2/10  Iteration 248/1780 Training loss: 2.4383 8.6865 sec/batch
Epoch 2/10  Iteration 249/1780 Training loss: 2.4372 8.0678 sec/batch
Epoch 2/10  Iteratio

Epoch 2/10  Iteration 353/1780 Training loss: 2.3159 9.7906 sec/batch
Epoch 2/10  Iteration 354/1780 Training loss: 2.3150 8.3333 sec/batch
Epoch 2/10  Iteration 355/1780 Training loss: 2.3139 8.4135 sec/batch
Epoch 2/10  Iteration 356/1780 Training loss: 2.3130 8.7149 sec/batch
Epoch 3/10  Iteration 357/1780 Training loss: 2.2007 8.3389 sec/batch
Epoch 3/10  Iteration 358/1780 Training loss: 2.1526 8.3493 sec/batch
Epoch 3/10  Iteration 359/1780 Training loss: 2.1414 8.7806 sec/batch
Epoch 3/10  Iteration 360/1780 Training loss: 2.1368 8.1400 sec/batch
Epoch 3/10  Iteration 361/1780 Training loss: 2.1358 8.5029 sec/batch
Epoch 3/10  Iteration 362/1780 Training loss: 2.1298 8.3272 sec/batch
Epoch 3/10  Iteration 363/1780 Training loss: 2.1291 8.3721 sec/batch
Epoch 3/10  Iteration 364/1780 Training loss: 2.1295 8.5232 sec/batch
Epoch 3/10  Iteration 365/1780 Training loss: 2.1316 8.1282 sec/batch
Epoch 3/10  Iteration 366/1780 Training loss: 2.1312 8.1639 sec/batch
Epoch 3/10  Iteratio

Epoch 3/10  Iteration 470/1780 Training loss: 2.0427 8.0368 sec/batch
Epoch 3/10  Iteration 471/1780 Training loss: 2.0418 8.2209 sec/batch
Epoch 3/10  Iteration 472/1780 Training loss: 2.0408 8.1683 sec/batch
Epoch 3/10  Iteration 473/1780 Training loss: 2.0402 8.1019 sec/batch
Epoch 3/10  Iteration 474/1780 Training loss: 2.0395 8.2756 sec/batch
Epoch 3/10  Iteration 475/1780 Training loss: 2.0388 8.1031 sec/batch
Epoch 3/10  Iteration 476/1780 Training loss: 2.0381 8.2379 sec/batch
Epoch 3/10  Iteration 477/1780 Training loss: 2.0376 7.9707 sec/batch
Epoch 3/10  Iteration 478/1780 Training loss: 2.0368 8.1198 sec/batch
Epoch 3/10  Iteration 479/1780 Training loss: 2.0359 8.5529 sec/batch
Epoch 3/10  Iteration 480/1780 Training loss: 2.0355 9.1298 sec/batch
Epoch 3/10  Iteration 481/1780 Training loss: 2.0348 8.5618 sec/batch
Epoch 3/10  Iteration 482/1780 Training loss: 2.0339 8.2578 sec/batch
Epoch 3/10  Iteration 483/1780 Training loss: 2.0334 8.7328 sec/batch
Epoch 3/10  Iteratio

...

Epoch 8/10  Iteration 1283/1780 Training loss: 1.4731 7.6007 sec/batch
Epoch 8/10  Iteration 1284/1780 Training loss: 1.4717 7.6335 sec/batch
Epoch 8/10  Iteration 1285/1780 Training loss: 1.4698 7.6528 sec/batch
Epoch 8/10  Iteration 1286/1780 Training loss: 1.4692 7.6257 sec/batch
Epoch 8/10  Iteration 1287/1780 Training loss: 1.4686 7.5851 sec/batch
Epoch 8/10  Iteration 1288/1780 Training loss: 1.4692 7.4419 sec/batch
Epoch 8/10  Iteration 1289/1780 Training loss: 1.4687 7.6059 sec/batch
Epoch 8/10  Iteration 1290/1780 Training loss: 1.4678 7.4481 sec/batch
Epoch 8/10  Iteration 1291/1780 Training loss: 1.4676 7.6316 sec/batch
Epoch 8/10  Iteration 1292/1780 Training loss: 1.4667 7.5927 sec/batch
Epoch 8/10  Iteration 1293/1780 Training loss: 1.4660 7.5208 sec/batch
Epoch 8/10  Iteration 1294/1780 Training loss: 1.4653 7.6595 sec/batch
Epoch 8/10  Iteration 1295/1780 Training loss: 1.4652 7.6297 sec/batch
Epoch 8/10  Iteration 1296/1780 Training loss: 1.4654 7.6403 sec/batch
Epoch 

Epoch 8/10  Iteration 1398/1780 Training loss: 1.4482 8.9923 sec/batch
Epoch 8/10  Iteration 1399/1780 Training loss: 1.4482 9.6625 sec/batch
Epoch 8/10  Iteration 1400/1780 Training loss: 1.4481 11.7364 sec/batch
Validation loss: 1.3213689 Saving checkpoint!
Epoch 8/10  Iteration 1401/1780 Training loss: 1.4488 8.9872 sec/batch
Epoch 8/10  Iteration 1402/1780 Training loss: 1.4488 8.1109 sec/batch
Epoch 8/10  Iteration 1403/1780 Training loss: 1.4488 8.6652 sec/batch
Epoch 8/10  Iteration 1404/1780 Training loss: 1.4487 8.6523 sec/batch
Epoch 8/10  Iteration 1405/1780 Training loss: 1.4483 8.8408 sec/batch
Epoch 8/10  Iteration 1406/1780 Training loss: 1.4483 10.4774 sec/batch
Epoch 8/10  Iteration 1407/1780 Training loss: 1.4485 9.3659 sec/batch
Epoch 8/10  Iteration 1408/1780 Training loss: 1.4485 9.6369 sec/batch
Epoch 8/10  Iteration 1409/1780 Training loss: 1.4484 8.4372 sec/batch
Epoch 8/10  Iteration 1410/1780 Training loss: 1.4483 8.9518 sec/batch
Epoch 8/10  Iteration 1411/17

Epoch 9/10  Iteration 1513/1780 Training loss: 1.4120 7.7982 sec/batch
Epoch 9/10  Iteration 1514/1780 Training loss: 1.4120 7.7862 sec/batch
Epoch 9/10  Iteration 1515/1780 Training loss: 1.4116 7.9417 sec/batch
Epoch 9/10  Iteration 1516/1780 Training loss: 1.4115 7.8423 sec/batch
Epoch 9/10  Iteration 1517/1780 Training loss: 1.4110 7.9415 sec/batch
Epoch 9/10  Iteration 1518/1780 Training loss: 1.4106 8.5948 sec/batch
Epoch 9/10  Iteration 1519/1780 Training loss: 1.4102 8.4305 sec/batch
Epoch 9/10  Iteration 1520/1780 Training loss: 1.4103 8.6382 sec/batch
Epoch 9/10  Iteration 1521/1780 Training loss: 1.4100 8.5599 sec/batch
Epoch 9/10  Iteration 1522/1780 Training loss: 1.4096 8.5528 sec/batch
Epoch 9/10  Iteration 1523/1780 Training loss: 1.4092 8.3250 sec/batch
Epoch 9/10  Iteration 1524/1780 Training loss: 1.4087 8.4673 sec/batch
Epoch 9/10  Iteration 1525/1780 Training loss: 1.4086 8.4590 sec/batch
Epoch 9/10  Iteration 1526/1780 Training loss: 1.4083 8.3846 sec/batch
Epoch 

Epoch 10/10  Iteration 1628/1780 Training loss: 1.3841 8.3598 sec/batch
Epoch 10/10  Iteration 1629/1780 Training loss: 1.3824 9.1009 sec/batch
Epoch 10/10  Iteration 1630/1780 Training loss: 1.3829 8.4138 sec/batch
Epoch 10/10  Iteration 1631/1780 Training loss: 1.3828 8.3981 sec/batch
Epoch 10/10  Iteration 1632/1780 Training loss: 1.3828 8.8034 sec/batch
Epoch 10/10  Iteration 1633/1780 Training loss: 1.3822 8.6465 sec/batch
Epoch 10/10  Iteration 1634/1780 Training loss: 1.3810 8.4741 sec/batch
Epoch 10/10  Iteration 1635/1780 Training loss: 1.3813 8.2792 sec/batch
Epoch 10/10  Iteration 1636/1780 Training loss: 1.3818 8.5069 sec/batch
Epoch 10/10  Iteration 1637/1780 Training loss: 1.3814 8.4735 sec/batch
Epoch 10/10  Iteration 1638/1780 Training loss: 1.3813 8.4214 sec/batch
Epoch 10/10  Iteration 1639/1780 Training loss: 1.3802 8.4450 sec/batch
Epoch 10/10  Iteration 1640/1780 Training loss: 1.3789 8.4533 sec/batch
Epoch 10/10  Iteration 1641/1780 Training loss: 1.3773 8.4073 se

Epoch 10/10  Iteration 1742/1780 Training loss: 1.3625 5.7827 sec/batch
Epoch 10/10  Iteration 1743/1780 Training loss: 1.3628 5.7607 sec/batch
Epoch 10/10  Iteration 1744/1780 Training loss: 1.3628 5.7539 sec/batch
Epoch 10/10  Iteration 1745/1780 Training loss: 1.3626 5.9382 sec/batch
Epoch 10/10  Iteration 1746/1780 Training loss: 1.3627 5.7802 sec/batch
Epoch 10/10  Iteration 1747/1780 Training loss: 1.3626 5.8434 sec/batch
Epoch 10/10  Iteration 1748/1780 Training loss: 1.3627 5.7902 sec/batch
Epoch 10/10  Iteration 1749/1780 Training loss: 1.3627 5.8456 sec/batch
Epoch 10/10  Iteration 1750/1780 Training loss: 1.3629 5.8517 sec/batch
Epoch 10/10  Iteration 1751/1780 Training loss: 1.3630 5.7075 sec/batch
Epoch 10/10  Iteration 1752/1780 Training loss: 1.3628 5.8561 sec/batch
Epoch 10/10  Iteration 1753/1780 Training loss: 1.3624 5.9057 sec/batch
Epoch 10/10  Iteration 1754/1780 Training loss: 1.3623 6.1223 sec/batch
Epoch 10/10  Iteration 1755/1780 Training loss: 1.3623 5.8371 se

In [20]:
tf.train.get_checkpoint_state('checkpoints/anna')

model_checkpoint_path: "checkpoints/anna/i1780_l512_1.256.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i200_l512_2.436.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i400_l512_1.992.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i600_l512_1.758.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i800_l512_1.604.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i1000_l512_1.485.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i1200_l512_1.407.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i1400_l512_1.348.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i1600_l512_1.306.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i1780_l512_1.256.ckpt"

## Sampling

Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.



In [21]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [22]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    prime = "Far"
    samples = [c for c in prime]
    model = build_rnn(vocab_size, lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.preds, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.preds, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

In [24]:
checkpoint = "checkpoints/anna/i200_l512_2.436.ckpt"
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/anna/i200_l512_2.436.ckpt
Faru/M/Yx/Y?SMN/0Y0/xS/MxNAuMz/YM/xuA/MxuTZ/STZ/MYA/MxN/STZ/MuMxN/ST/SMN/MYM/MN/xN/MxN?NANA/S0MxSZ/0ST/MxN/MxN/MxN/0uAMN/ST/xY0/0S0/MxN/MxuT/MS0N0u'Z/ST/MY0/MxN/0NM/ST/MY/0N0ST/xN/7YAN0/ST/xuA/xN0S0/S0/YT/MxN/MYANA0uTTZ/STz/7ST0MSTM/7Y/7ST'N/MSTZ`STZ/SMYTMx/MY0/xS00S0Zm/-SZ/0NMuM/xN/ST0/xSA/7STMN/MxuM/MxY0/xN0S0Z/SMN/0YAN/MxN/MY0/S0M/ST/xNAN/xuTzm/xN/0SMN0/SZuM/xN/7YAuTz/MN/STMN/7YANTTzN/0SMxN/7SANA/MxuANA/STMxN/0Y/ST/YASTZ/STz/MxNA/STZ/MuTz/MxN`MSA/xxN/MSA/0N/MxN/ST0/YT/MYT/xN/7ST/xN/MxNA/xN/0NASTz/MxN/MxS0/MxS/xN0/7YA/STZ/0S0MuTTz/SM/xuT/MSAN/0u0M/uT/MxSANASTZ/0ST/xN/ST/MYTZ/ST/xN/0NSM/xYA/S0/MxN/MxNA/SMxST/ST/SMN/MxNAuTT/xuM/xN/S'NA/YMxN/7N/MxY/S''/MxN/MYA/7xN/7xNA/MYP/'N/xS0/MN/0ST0/uT/MxS/MxN/MxS0/SAN/xS0m/-uM/xx/?xN/xxu0/xSZ/0YAN/MSM/xuM/xu0MN/Mx/SMxNTZ/STZ/0SZ/7YT/xN/xN/MSAuM/xNM/xS0/MxS0/0YMxN/MY/xYM/xx/0N/xN0uTT/YT/MxN/7ST0/7N00/xN/MxN/STz/Y0/MxN/MxN/0YA/SM/xS/MxS/7ST0/xNAu0Z/xN/MxNASA/0YAN0/xY/0u?

In [25]:
checkpoint = "checkpoints/anna/i800_l512_1.604.ckpt"
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/anna/i800_l512_1.604.ckpt
FarN`xSZ/0SuZ/S''/MxN/?YAN/Yf/MxN/UY0uYT/Yf/MxN/0SuZo``1'NbNt/1ANbSTZAYOuMBx`7S0/TYM/xNA/0MYUU'N/Yf/S/Mu?N/YT/S/0SuZ`Yf/MxN/7S0/MxN/0S7MNA/Yf/uT/Yff/STZ/S''/xu0/7S0/0xu''m/0MuTz'N/xSON/STZ/MxN/0MuAZ/S/?STMNA/MxN/BYP'Z/STA/0NNMuMN/MxN/0Y?N/STZ/0Suzm/STZ`MxN/0MNTz/S/0SuMo``KxN/USuTMuTz/Yf/MxN/0NSMMNAm/STZ/MY/MxN/USAANZo/pxN/7S0/TYM/xNA/0YT/MxN/UYAu0M/MxN`0MNA0uMN/xNA/Mxu0`7S0/MxSM/0SuZm/STZ/xSZ`0NSMxNA/MY/xu0/xN/7uTN/STZ/xSZ/-NNT/uT/MxN`ANS?YT/0xN/7S0/SM/MxN/ZuZNTM/7Y?/MxN/7YAnN/MxN?o``8$M40/TNMMANZ/SM/S''/MxN/0Mu''o/$/?S0M/u0/STZ/STZ`0MNUUSA0/STZ/7uMx/MxN/?YTN/MxN/?STMN/7YP'Z/0xN/0S7/tYP/ZAu0BNm/7xSM/0xN/ZY7T/S/ZAuTzm/-PM/ST'/MxNuz/S''/xu0/0MSM/STZ/S/0MANSz/MY/-N/0NA?NT/uT/MxN`UYA0N00o/1TZ`xNA/MS'nNZ/xu?/uT/MxN`UYTMNA0m/STZ/MxN/?STZ/YPM`MxN/?STMuYA/7NAN/SM/MxN/0YATm/0SuZ/MxN?N/S/?SA0uYT/MxSM/0xN/7S0/0YYT/xNA/S/0S?N/Yf/xu0/0USuA/Yf/MxN/BYTMNA00/Yf/uM/7xYM`xN/ZY7SAM/Yf/xu?/Yf/S0/MxN/0YTNm/STZ/0xN/0Y?U/Yf/0NN?N

In [26]:
checkpoint = "checkpoints/anna/i1400_l512_1.348.ckpt"
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/anna/i1400_l512_1.348.ckpt
FarN`MxN/xS0/-NzuT/0UuMuYTS''tm/STZ/0xN/BS''NZ/xNA/0SuZm/0xN/xSZ/0NNT/xu?/uT/MxN/UAN0MSMN/MY/MxN/0YPM/Yf/xNA/NtN0/STZ`MxN/US00/S0/xN/7S0/TYM/MY/0NN/uMo/rN/7S0/0NN?NZ/SM/MxN/0Y?Nm/7uMx/xNA/xu0/7xuBxo``8$TZNZm/$40/S/UNTM'tm/STZ/$/ZY/MxN/BxNSAMm/$/7STMuTz/MY/xNSA/S/'YONo/wPM/tYP/nTY7/MY/-Nm8/0SuZ/vNOuTm/8STZ/S/?SA0xuYT/MxN/0PUUYAMuYTo/wPM/uT/STt/Yf/xu?/SM/MxN`0S?N/7uMx`xNA/SMBxN?UNAo8``8axY/0PAUY00/uM/7N''m/$/ZYT4M/-N/0Y?NMxuTzo8``8$/ZST4M/nNUM/tYPm/7xNAN/tYP/7NAN/MY/?N/uT/MxN/0Y?N/MxYPzxM/MxN/0Y?N/MY/?YAN/Yf/MxN0N/7uMx/tYPA/0NTZ/xu?o/wPM/$4''/-N/ZN'NTMuON/YPMm/-PM/MxNAN/7S0/S''/MxN/fuzxM/MY`-N/STZ/STzNA/uT/MxN/UYAMAN00o8``8$4ON/TYM/-N/S7SuZ/MY/MY/xSON/S-'NNZ/uM/7uMx/?u0xSMNm8/0xN/0SuZm/8STZ/S/'YTz/MxN/0Y?N/YTN/7uMx/MxN/?uTZ/Yf`uM/S/BYT0PUMuYT/MxSM/uM/MS'nNZ/Yf/MxN/UNMfNA/SM`S''/Yf/MxN/?P0M/S/BYTOuMuYT/7uMx/MxN/UAuTBN00/STZ/MxSM/7S0/0xN/0S7/MxSMm/STZ/STZ/0NAN/MY/MxN/0S7N/YT/xNA/MS'nm/7uMx/xu0/0P-0uBuS'/MASTzN

In [27]:
checkpoint = "checkpoints/anna/i1780_l512_1.256.ckpt"
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/anna/i1780_l512_1.256.ckpt
FaruMx/MxN/?Y0M/SAYPzx`uT/0Y?NMu?Nm/7xuBx/xN/0S7/MxN/UANONTM/MY/0NN/xu?/S0/SM/YTBNo/KxNT`7NAN/0xY7uTz/xNA/MY/MxN/0NAfSMuYTm/STZ/0S7/S/0uzTuTz/MY/ST/NONTuTzo``8$/7N0M/0MASTzNm/MY''/?t/MxNuAo/axSM/ZAN00/MxN/BxSTzN/Yf/tYPA/0YP'0m/STZ/0xN/ZYT4M/-N'N/SM`uM/STZ/Zu0BP00uYT/MxSM/tYP/7YP'Z/xSON/-NNT/0PTZYTZNA0o8``8axSM/u0/MxN/BYP0M/MxNtJ/rSON/$/ZYT4M/0MNU/MxN?m/-PM/MxN/?STm/STZ/MxN/0NNMNZ`u0/0Y/MY/-Nm/S/ZYYA/S0/MxNt/0xSTM/MxNAN/MY/ZY/STt/0StuTzm/-t/MxNAN/7NAN`0YT/YMxNAo8``8KxSM40/STt7Stm/STZ/$/ZYT4M/nTY7/S''/MxN/?ST/Yf/S''/Yff/MxNuN/MY/-N/MxN`?YANtm8/S00NZ/vNOuTm/0Y/SM/YTBN/SM/MxN/BxP'Z/-t/Mxu0/0uzN0/7NANm/STZ/MxN/BYABuUM`Yf/xu0/BYTBNTMNZ/xu?/STZ/MxSM/MxN/0S?N/0YT/STt/xSTZ0m/7xY?/xN/7S0/BYTONA0SMuYT/MxSM`xN/0xYP'Z/xSON/0PAN/MxN/UAuONA0/STZ/xu0/fAYNM0m/STZ/xN/xSZ/-NNT/0Y?NMxuTz/STZ`BYTOuTBuTzm/MxN/0S?N/0MS''/YA/SMMSBxNZ/MxN/0S?N/7xSM/xN/xSZ/TYM/BxSTzNZ`MY/xu?m/xN/0SM/xSZ/-ANSMuTz/xu0/xSAZm/STZ/xu0/BST/0NTm/MxNAN/7S0